# Finetuning a model on labeled counterspeech data (CONAN)

CONAN (COunter NArratives through Nichesourcing): A dataset containing counterspeech responses to hate speech. Data is French, italian and English

https://github.com/marcoguerini/CONAN

https://aclanthology.org/P19-1271.pdf

In [1]:
!pip install transformers datasets pandas scikit-learn torch


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

conan = pd.read_csv('/Users/idahelenedencker/Desktop/STANDBY_Ida/Creating dataset of reference tweets/CONAN.csv')
conan

# Count how many unqie instances of each
print(conan['counterSpeech'].nunique())
print(conan['hateSpeech'].nunique())

6803
856


In [3]:
# Combine hate speech and counter speech into a single DataFrame
data = {
    'text': conan['hateSpeech'].tolist() + conan['counterSpeech'].tolist(),
    'label': [0] * len(conan['hateSpeech']) + [1] * len(conan['counterSpeech'])}

# Make into df
df_prepared = pd.DataFrame(data)

# Drop duplicates
df_prepared.drop_duplicates(inplace=True)

# Count each label
df_prepared.value_counts(df_prepared['label'])


label
1    6803
0     856
Name: count, dtype: int64

In [5]:
df_prepared

,text,label
0,"According to a recent OFSTED report, a school ...",0
1,In Birmingham there is a school where girls an...,0
2,A school in Birningham is still separating gir...,0
12,The police and politicians are covering up the...,0
13,Muslims grooming gangs are protected by the go...,0
...,...,...
29967,Italy unfortunately is full of unveiled women ...,1
29968,"Nessuna donna accetta di essere picchiata, non...",1
29971,"No woman agrees to be beaten, be unable to bre...",1
29972,Leggere commenti di donne capaci di scrivere t...,1


In [4]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

# Load the dataset
df = df_prepared

# Split into training and validation sets (90% train, 10% validation)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Prepare the data for Hugging Face
train_texts = train_df['text'].tolist()
train_labels = train_df['label'].tolist()

val_texts = val_df['text'].tolist()
val_labels = val_df['label'].tolist()

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"  # You can replace this with other models like "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Create a custom Dataset class
class CounterspeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create PyTorch datasets and dataloaders
train_dataset = CounterspeechDataset(train_encodings, train_labels)
val_dataset = CounterspeechDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define the training arguments with modified paths
training_args = TrainingArguments(
    output_dir='./finetuning/results',          # output directory for model checkpoints
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./finetuning/logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluate after each epoch
    save_strategy="epoch",           # Save the model after each epoch
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the fine-tuned model and tokenizer in the 'finetuning' directory
model.save_pretrained("./finetuning/model")
tokenizer.save_pretrained("./finetuning/tokenizer")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will b

Epoch,Training Loss,Validation Loss
1,0.242900,0.168110
2,0.098400,0.129995
3,0.005300,0.158067


Evaluation results: {'eval_loss': 0.15806695818901062, 'eval_runtime': 14.2365, 'eval_samples_per_second': 53.805, 'eval_steps_per_second': 3.372, 'epoch': 3.0}


('./finetuning/tokenizer/tokenizer_config.json',
 './finetuning/tokenizer/special_tokens_map.json',
 './finetuning/tokenizer/vocab.txt',
 './finetuning/tokenizer/added_tokens.json',
 './finetuning/tokenizer/tokenizer.json')

In [7]:
# Test the model on a sentence

input_text = "i like horses"
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=128)

# Move inputs to CPU
inputs = {key: val.cpu() for key, val in inputs.items()}

# Move the model to CPU
model.to("cpu")

# Perform inference
outputs = model(**inputs)
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1)
print(f"Predicted class: {predicted_class.item()}")


Predicted class: 1


In [8]:
# Apply the model to the pairs data

#load in the translated data
#Load in
dtype_dict_all = {
    'conversation_id': 'object',
    'id': 'object',
    'author_id': 'object',
    'referenced_tweets_id': 'object',
    'in_reply_to_user_id': 'object',
    'PNR': 'object'
}

pairs = pd.read_csv('/Users/idahelenedencker/Desktop/STANDBY_Ida/Creating dataset of reference tweets/w_translated_small.csv', dtype=dtype_dict_all)



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the saved model and tokenizer
model_name = "./finetuning/model"
tokenizer_name = "./finetuning/tokenizer"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)


# Tokenize the new data
encodings = tokenizer(pairs['translated'].tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')

#create a data loader
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

# Create the dataset and dataloader
new_dataset = CustomDataset(encodings)


new_loader = DataLoader(new_dataset, batch_size=16, shuffle=False)

#make predictions and evaluate
from tqdm import tqdm

model.eval()  # Set the model to evaluation mode

predictions = []
with torch.no_grad():
    for batch in tqdm(new_loader):
        inputs = {key: val.to(model.device) for key, val in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())

# Add predictions to the DataFrame
pairs['finetune_predictions'] = predictions

pairs


In [ ]:
# Inspect the results

# How many of each class
print(pairs.value_counts(pairs['finetune_predictions']))

#print
finetune_yes= pairs[pairs['finetune_predictions'] == 1]
finetune_no= pairs[pairs['finetune_predictions'] == 0]


text_to_print = finetune_yes['translated'].head(20).tolist()
print('Marked as counterspeech:')
print(text_to_print) 

text_to_print = finetune_no['translated'].head(20).tolist()
print('Not marked as counterspeech:')
print(text_to_print) 


# Marks almost all as counterspeech which is not good, it's quite clear that what the model has used as label 0's (hatefull speech) in the training process contain hateful comments on islam, and hence what is labeled as 0's here contain mainly hateful/harsh language and/or islamic aspects



# (maybe) Finetuning the bestperforming huggingface counterspeech classifier model on a labeled danish dataset (i can label some?)

Dont know if this is problamatic, since the model will be finetuned on (a sample of) the same labeled dataset that i will afterwards apply the finetuned model to? 

An idea could be to remove the labeled sample from the test dataset. 